### Abhängigkeiten Importieren

In [53]:
import ipycanvas
import ipywidgets
import numpy
import math

### Globale Konstanten

In [54]:
BOARD_SIZE = 8

NONE = 0
BLACK = 1
WHITE = 2

SHOW_FRONTIER = False
SHOW_POSSIBLE_MOVES = True

### Game State Klasse

Repäsentiert den Spielzustand

In [55]:
class GameState:
    def __init__(self):
        self.board = numpy.zeros((8, 8), dtype=numpy.int8)
        self.board[3, 3] = WHITE
        self.board[3, 4] = BLACK
        self.board[4, 3] = BLACK
        self.board[4, 4] = WHITE
        self.frontier = {(2,2),(2,3),(2,4),(2,5),
                         (3,2),(3,5),(4,2),(4,5),
                         (5,2),(5,3),(5,4),(5,5)}
        self.turn = BLACK

In [56]:
directions = {(-1,-1),(0,-1),(1,-1),(-1,0),(1,0),(-1,1),(0,1),(1,1)}

In [57]:
def opposite_player(player):
    return {BLACK: WHITE, WHITE: BLACK}[player]

In [58]:
class InvalidMoveException(Exception):
    pass

In [59]:
def make_move(state, pos):
    if pos not in state.frontier:
        raise InvalidMoveException
        
    possible_directions = adjacent_opposite_color_directions(state, pos[0], pos[1], state.turn)

    next_turn = state.turn
    for direction in possible_directions:
        if is_valid_directional_move(state, pos[0], pos[1], direction[0], direction[1], state.turn):
            next_turn = opposite_player(state.turn)
            convert_adjacent_cells_in_direction(state, pos[0], pos[1], direction[0], direction[1], state.turn)

    if next_turn != state.turn:
        state.board[pos] = state.turn
        update_frontier(state, pos[0], pos[1])
        if can_move(state, next_turn):
            state.turn = next_turn
    else:
        raise InvalidMoveException()

In [60]:
def is_game_over(state):
    return not can_move(state, BLACK) and not can_move(state, WHITE)

In [61]:
def get_winner(state):
    black_disks = count_disks(state, BLACK)
    white_disks = count_disks(state, WHITE)
    if black_disks > white_disks:
        return BLACK
    if white_disks > black_disks:
        return WHITE
    else:
        return NONE

In [62]:
def get_possible_moves(state, player):
    possible_moves = []
    for (row, col) in state.frontier:
        for direction in adjacent_opposite_color_directions(state, row, col, player):
            if is_valid_directional_move(state, row, col, direction[0], direction[1], player):
                possible_moves.append((row, col))
                break
    return possible_moves

In [63]:
def can_move(state, player):
    return len(get_possible_moves(state, player)) != 0

Looks up to a possible of 8 directions surrounding the given move. If any of the
move's surrounding cells is the opposite color of the move itself, then record
the direction it is in and store it in a list of tuples [(rowdelta, coldelta)].
Return the list of the directions at the end.

In [64]:
def adjacent_opposite_color_directions(state, row, col, turn):
        dir_list = []
        for rowdelta in range(-1, 2):
            if not 0 <= row+rowdelta < 8:
                continue
            for coldelta in range(-1, 2):
                if not 0 <= col+coldelta < 8:
                    continue
                if state.board[row + rowdelta, col + coldelta] == opposite_player(turn):
                    dir_list.append((rowdelta, coldelta))
        return dir_list

Given a move at specified row/col, checks in the given direction to see if
a valid move can be made. Returns True if it can; False otherwise.
Only supposed to be used in conjunction with _adjacent_opposite_color_directions()

In [65]:
def is_valid_directional_move(state, row, col, rowdelta, coldelta, player):
        current_row = row + rowdelta
        current_col = col + coldelta

        last_cell_color = opposite_player(player)

        while True:
            if not (0 <= current_row < 8 and 0 <= current_col < 8):
                break
            if state.board[current_row, current_col] == NONE:
                break           
            if state.board[current_row, current_col] == player:
                last_cell_color = player
                break

            current_row += rowdelta
            current_col += coldelta
            
        return last_cell_color == player

In [66]:
def count_disks(state, player):
    return numpy.count_nonzero(state.board == player)

In [67]:
def get_player_string(player):
    return {BLACK: 'Black', WHITE: 'White', NONE: 'Nobody'}[player]

In [68]:
def convert_adjacent_cells_in_direction(state, row, col, rowdelta, coldelta, player):
    ''' If it can, converts all the adjacent/contiguous cells on a turn in
        a given direction until it finally reaches the specified cell's original color '''
    if is_valid_directional_move(state, row, col, rowdelta, coldelta, player):
        current_row = row + rowdelta
        current_col = col + coldelta
        
        while state.board[current_row, current_col] == opposite_player(player):
            state.board[(current_row, current_col)] = player
            current_row += rowdelta
            current_col += coldelta

In [69]:
def update_frontier(state, row, col):
        for current_row in range(row-1, row+2):
            if not 0 <= current_row < 8:
                continue
            for current_col in range(col-1, col+2):
                if not 0 <= current_col < 8:
                    continue
                if state.board[current_row, current_col] == NONE:
                    state.frontier.add((current_row, current_col))
        state.frontier.remove((row, col))

### Widgets Initialisieren

#### Canvas Widget Initialisieren

In [70]:
CELL_SIZE = 70

CANVAS_SIZE = BOARD_SIZE * CELL_SIZE

canvas = ipycanvas.MultiCanvas(2, width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas[0].fill_style = 'darkgreen'
canvas[0].stroke_style = 'black'
canvas[0].fill_rect(0, 0, CANVAS_SIZE, CANVAS_SIZE)
canvas[0].begin_path()
for i in range(BOARD_SIZE+1):
    pos = i * CELL_SIZE
    canvas[0].move_to(pos, 0)
    canvas[0].line_to(pos, CANVAS_SIZE)
    canvas[0].move_to(0, pos)
    canvas[0].line_to(CANVAS_SIZE, pos)
canvas[0].stroke()

#### Text Widgets initialisieren

In [71]:
score_lbl = ipywidgets.widgets.Label()

In [72]:
turn_lbl = ipywidgets.widgets.Label()

#### Output Widget initalisieren

In [73]:
output = ipywidgets.widgets.Output()

In [74]:
def display_board(state):
    update_output(state)
    display(canvas)
    display(score_lbl)
    display(turn_lbl)
    display(output)

In [75]:
def update_output(state):
    with ipycanvas.hold_canvas(canvas):
        canvas[1].clear()
        for ((x, y), val) in numpy.ndenumerate(state.board):
            if val == NONE:
                continue
            elif val == BLACK:
                canvas[1].fill_style = 'black'
            else:
                canvas[1].fill_style = 'white'
            canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 2.2, 0, 2 * math.pi)
            
        if SHOW_FRONTIER:
            for (x, y) in state.frontier:
                canvas[1].fill_style = 'gray'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)
        
        if SHOW_POSSIBLE_MOVES:
            for (x, y) in get_possible_moves(state, state.turn):
                if state.turn == BLACK:
                    canvas[1].fill_style = 'black'
                else:
                    canvas[1].fill_style = 'white'
                canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 6, 0, 2 * math.pi)
            
    score_lbl.value = f'Black Player : {count_disks(state, BLACK)} White Player : {count_disks(state, WHITE)}'
    if is_game_over(state):
        turn_lbl.value = f'{get_player_string(get_winner(state))} wins'
    else:
        turn_lbl.value = f'{get_player_string(state.turn)}s Move'

In [76]:
def mouse_down(x_px, y_px):
    if not is_game_over(state):
        with output:
            x = math.floor(x_px / CELL_SIZE)
            y = math.floor(y_px / CELL_SIZE)
            try:
                make_move(state, (x, y))
            except InvalidMoveException:
                print('Invalid Move')
            update_output(state)

canvas[1].on_mouse_down(mouse_down)

In [77]:
state = GameState()

In [78]:
display_board(state)

MultiCanvas(height=560, width=560)

Label(value='Black Player : 2 White Player : 2')

Label(value='Blacks Move')

Output()